In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta

**Vui lòng chuyển đổi sang Vnstock3** thế hệ mới (0.3.0.8) với câu lệnh: `pip install vnstock3 --upgrade`.
**Từ 1/1/2025, vnstock3 sẽ được cài đặt khi sử dụng cú pháp** `pip install vnstock` **thay cho Vnstock Legacy** hiện tại.
Xem chi tiết [chuyển đổi sang vnstock3](https://vnstocks.com/docs/tai-lieu/migration-chuyen-doi-sang-vnstock3).
Phiên bản **Vnstock Legacy (0.2.9.2.3)** bạn đang sử dụng **sẽ không được nâng cấp thêm.**
Từ 7/10/2024 Vnstock giới thiệu nhóm Facebook Cộng đồng Vnstock, tham gia thảo luận tại đây: https://www.facebook.com/groups/vnstock.official

In [2]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 100_000_000

In [3]:
VN30 = ['CII','CTD','CTG','DHG','DPM','EIB','FPT',
        'GAS','GMD','HDB','HPG','MBB','MSN','MWG','NVL',
        'PNJ','REE','ROS','SAB','SBT','SSI','STB','TCB',
        'VCB','VHM','VIC','VJC','VNM','VPB','VRE']

In [5]:
def calculate_indicators_macd(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [6]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [4]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [8]:
def simulate_investment_macd(ticker):
    try:
        data = vn.stock_historical_data(ticker, '2017-07-24', '2019-02-18', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators_macd(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+2 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            if current_date in buy_signals:
                # Invest 
                allocation = cash 
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['close'][i])
                            total_cost = shares_to_buy * data['close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            # Check if we need to sell due to sell signal
            current_price = data['close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [11]:
def backtest_macd(companies_vn30):
    results = []
    for company in companies_vn30:
        result = simulate_investment_macd(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [12]:
macd_res = backtest_macd(VN30)

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Error occurred for DPM: Out of bounds nanosecond timestamp: 9223545600000000000


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error occurred for ROS: cannot access local variable 'df' where it is not associated with a value


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

In [13]:
macd_res

,Company,Final Portfolio Value,Total Profit,Rate of Return
0,CII,77461670,-22538330,-22.538330
1,CTD,76660690,-23339310,-23.339310
2,CTG,136629010,36629010,36.629010
3,DHG,112218860,12218860,12.218860
4,EIB,115880620,15880620,15.880620
5,FPT,119976374,19976374,19.976374
6,GAS,110987800,10987800,10.987800
7,GMD,84305020,-15694980,-15.694980
8,HDB,78623370,-21376630,-21.376630
9,HPG,117107560,17107560,17.107560


In [14]:
macd_res.to_excel('decimalodd_res/macd.xlsx')

In [5]:
def calculate_indicators_bb(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [6]:
def bb_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    df.loc[
        (df['close'] <= df['Bollinger_low']) &
        (df['RSI'] <= RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] >= RSI_OVERBOUGHT) &
        (df['close'] >= df['Bollinger_high']), 'Signal'] = -1

    return df

In [7]:
def simulate_investment_bb(ticker):
    try:
        data = vn.stock_historical_data(ticker, '2017-07-24', '2019-02-18', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators_bb(data)
        data = bb_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+2 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            if current_date in buy_signals:
                # Invest 
                allocation = cash 
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['close'][i])
                            total_cost = shares_to_buy * data['close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            # Check if we need to sell due to sell signal
            current_price = data['close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [8]:
def backtest_bb(companies_vn30):
    results = []
    for company in companies_vn30:
        result = simulate_investment_bb(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [9]:
bb_res = backtest_bb(VN30)

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
A value is trying to be set on a copy of a DataFrame or Series throug

Error occurred for GAS: Out of bounds nanosecond timestamp: 9223545600000000000


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
A value is trying to be set on a copy of a DataFrame or Series throug

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error occurred for ROS: cannot access local variable 'df' where it is not associated with a value


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
A value is trying to be set on a copy of a DataFrame or Series throug

In [10]:
bb_res.to_excel('decimalodd_res/bb.xlsx')

In [22]:
OBV_PERIOD = 5

In [23]:
def calculate_indicators_obv(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

def strategy_obv(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI across 30 and OBV rise
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1

    #Sell Signals: RSI across 70 and OBV down 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1

    return df


In [24]:
def simulate_investment_obv(ticker):
    try:
        data = vn.stock_historical_data(ticker, '2017-07-24', '2019-02-18', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators_obv(data)
        data = strategy_obv(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+2 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            if current_date in buy_signals:
                # Invest 
                allocation = cash 
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['close'][i])
                            total_cost = shares_to_buy * data['close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            # Check if we need to sell due to sell signal
            current_price = data['close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [25]:
def backtest_obv(companies_vn30):
    results = []
    for company in companies_vn30:
        result = simulate_investment_obv(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [26]:
obv_res = backtest_obv(VN30)
obv_res.to_excel('decimalodd_res/obv.xlsx')

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
A value is trying to be set on a copy of a DataFrame or Series throug

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error occurred for ROS: cannot access local variable 'df' where it is not associated with a value


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
A value is trying to be set on a copy of a DataFrame or Series throug